In [1]:
# -*- encoding:utf-8 -*-
import pandas as pd
import base64
import urllib.parse
import re
import nltk
import urllib
import numpy as np
from sklearn import tree

In [2]:
def is_number(str):
    try:
        float(str)
        return True
    except:
        return False

def is_letter(str):
    return str.isalpha() and len(str) <=2

def is_base64_code(str):
    if len(str) < 20 :
        return False
    return  re.match('^[a-zA-Z0-9+-/=_]*[a-zA-Z0-9+-/=_]$', str)# re.match('[a-zA-Z0-9+/=]', str)

def is_base64_code_ex(str):
    if len(str) < 20:
        return False
    return  re.match('^[a-zA-Z0-9+-/=_ ]*[a-zA-Z0-9+-/=_ ]$', str)

def base64_url_decode(inp):
    # 通过url传输时去掉了=号，所以需要补上=号
    if len(inp) % 4 != 0:
        try:
            return base64.urlsafe_b64decode(str(inp)+'='* (4 - len(inp) % 4))
        except:
            return inp
    else:
        try:
            return base64.urlsafe_b64decode(str(inp))
        except:
            return inp

def base64_url_encode(inp):
    return base64.urlsafe_b64encode(str(inp)).rstrip('=')

def to_bytestring(s, enc='utf-8'):
    if s:
        if isinstance(s, str):
            return s
        else :
            return s.encode(enc)

In [3]:
def decode_file(file_name):
    params_list = []
    for line in open(file_name,'r',encoding="utf-8"):
        line=line.split(',',2)
        try:
            action=line[2].rstrip('\n')
        except:
            action=" "
        new_params = []
        if type(action) is float:
            action=str(action)
            params_list.append(action)
            continue
        rs = urllib.parse.urlparse('http://xxx?'+action)
        query_set = urllib.parse.parse_qs(rs.query)
        if len(query_set) == 0:
            new_params.append(action)
            params_list.append(new_params)
            continue
        for key, value in query_set.items():
            param = ''.join(value)
            if is_base64_code(param):
                new_param = base64_url_decode(param)
            else:
                new_param = ''
                if is_base64_code_ex(param) :
                    try:
                        for part_param in param.split(' '):
                            try:
                                temp = base64_url_decode(part_param)
                                new_param += str(temp)
                            except:
                                new_param += part_param
                    except:
                        new_param = param
                else :
                    new_param = param

            new_params.append(new_param)
            query_set[key] = new_param.split()
        params_list.append(new_params)
    return params_list
file_name = 'F:\contest\subject2_B\subject2_B\\filelist_B.txt'
test_list=decode_file(file_name)

In [4]:
def cut_word(params_list):
    doc_word_list = []
    for params in params_list:
        param_word_list = list()
        text = nltk.word_tokenize(str(params))
        for w in text:
            w = w.lower().lstrip(' ').rstrip(' ').replace("u'", ' ')
            if w[0] == "'":
                w = w[1:]
            w = w.lstrip(' ').rstrip(' ')
            if (is_number(w) == False and len(w) < 50 and is_letter(w) == False and len(w) != 0):
                param_word_list.append(w)
        doc_word_list.append(param_word_list)

    return doc_word_list
doc_word_list = cut_word(test_list)


In [13]:
def get_doc_features_matrix(doc_word_list, features):
    rows = len(doc_word_list)
    cols = len(features)
    doc_features_matrix = np.zeros((rows, cols))
    for i in range(1, rows):
        for j in range(1, cols):
            doc_features_matrix[i][j] = doc_word_list[i].count(features[j])
    return doc_features_matrix

In [14]:
features=['fclose', 'script_name', 'filestools', 'query', 'rewriterule', 'rewritecond', 'charset=', 'multipart/form-data', '/wwwroot/dwyshw11/wwwroot/home.php', '-|', 'windows-1251', '@', '/home2/risquemag/public_html/', "\\'php_self\\", 'charset=utf-8', 'host_name.', 'h\\x88\\xae\\xef\\xee\\xa3', 'content_mb', '--', 'die', 'host_name', 'execut', '/wwwroot/dnwcomcn//jiaozhuo.php', 'rewritebase', 'domain', ';', 'public', 'netstat', '[', "\\'posix_getegid\\", 'ini_set', '{', 'index.php', 'substr', '&', 'document_root', 'doctype', 'rewriterule', 'root', 'tar', 'strlen', '127.0.0.1', '!', 'end', 'php_uname', 'ping', 'foreach', 'xhtml', '-an', 'http', 'description', 'urldecode', 'enctype', 'charset=gb2312', 'opts', 'delfile', 'echo', 'php_config.php', 'remote_server.', '.substr', 'fwrite', 'set_magic_quotes_runtime', '<', '.php', 'eval', 'set_time_limit', 'execute', 'gethttppage', 'ifmodule', 'filesman', '|', 'meta', 'loginlogin', 'print', 'eval', 'wscript', 'http-equiv=', 'gb2312', 'xmlns=', 'echo', 'get_current_user', 'array', 'mkdir', 'windows', 'white', 'google|aol|yahoo|msn|search|bing|seznam|seznam', 'filesize', 'function_exists', 'shell', 'jstephens', 'para', 'fires', 'path_translated', 'robots.txt', "_p'.'os", 'melissacalhoun', 'ie=edge', 'path_translated', 'and', 'response.write', 'content_mb=gethttppage', '^', 'phpfunc', '-', '=', '-zcvf', 'begin', 'content=', 'net', 'mod_rewrite.c', 'content_mb=file_get_contents', '\\\\\\\\\\\\', 'buf=', 'dom', 'url', '}', 'keywords', 'mkfile', 'parameters', 'title', '(', '163.com', '_server', 'get', 'server_name', 'query_st', 'transitional', '.htaccess', 'view', 'posix_getpwuid', 'set', 'is_dir', '#', 'wordpress', 'url=', 'c1sall', 'jquery.js', 'base64_decode', 'php.exe', 'remote_server', 'ifmodule', 'query_string', 'http', 'login', 'dirname', 'script_filename', 'chmod', 'reg_type', 'array_map', 'usr', 'cmd', '>', 'add', 'host', 'main', 'downdb', 'rename', 'http_user_agent', 'systeminfo', 'base64_', 'index.php', ')', 'prc', 'html', 'http', 'set_time_limit', 'user', 'function', '/*', 'str_replace', 'rewritecond', 'post', 'base64_decode', 'touch', 'date_default_timezone_set', '$', 'location', 'utf-8', 'php_self', 'request_filename', 'web.config', 'fires', 'text/html', 'header', 'edit', 'rewriterule', 'posix_geteuid', 'hunter', 'header', 'for', 'http_host', '?', 'run', 'isset', 'range', 'content-type', 'visible', 'base64_decode', 'fopen', 'rewritecond', '.*', 'display_errors', ']', '*', 'wordpress', ':', 'head', 'pwd', 'eval', 'buf', '^index\\\\.php', 'edoced_46esab', 'rewriteengine', '/bin/sh', 'trim', 'fread', '%', 'http_referer', 'x-ua-compatible', 'str_replace', 'eval', 'downfil']

test_data=get_doc_features_matrix(doc_word_list, features)


In [22]:
def get_doc_features_matrix(doc_word_list, features):
    rows = len(doc_word_list)
    cols = len(features)
    doc_features_matrix = np.zeros((rows, cols))
    for i in range(1, rows):
        for j in range(1, cols):
            doc_features_matrix[i][j] = doc_word_list[i].count(features[j])
    return doc_features_matrix

In [23]:
file_name = 'F:\contest\subject2_sample\subject2_sample.txt'
params_list,y= decode_file(file_name)
doc_word_list = cut_word(params_list)


In [24]:
features=['c=', '\\\\\\\\freehost\\\\\\\\hqwkg293\\\\\\\\web\\\\\\\\index.php', 
 'fclose', 'script_name', 'filestools', 'query', '\\\\\\\\php_point\\\\\\\\include\\\\\\\\', '\\xfb\\xdb', '\\r\\nrewriterule',
 '\\r\\nrewritecond', 'evjonpzr', 'charset=', 'multipart/form-data', '/wwwroot/dwyshw11/wwwroot/home.php', 'x673a', '-|', 'windows-1251', 
 '\\x98\\xd0\\xe1e\\x9es', "1'='", '@', '/home2/risquemag/public_html/', "\\'php_self\\", 'charset=utf-8', 'host_name.', 'h\\x88\\xae\\xef\\xee\\xa3', '//www.w3.org/1999/xhtml', '//news.nikl.pw:88/1/', 'content_mb', 'z\\xfe\\x1b', '--', 'die', 'x53f0', 'host_name', 'execut\\\\00', '/wwwroot/dnwcomcn//jiaozhuo.php', 'gonzalesdavid', '/alidata/www/home/htdocs/site/css/ya.ph', 'ww\\x0e\\x95\\x10', '\\xe4\\xb8\\x8a\\xe4\\xbc\\xa0', '\\xdc\\x96\\xe7^\\xb3\\xf8\\xec\\xfe', '\\xdd\\xa3\\x0b', '\\x14\\x10a\\x10', '\\r\\nrewritebase', '53=47', 'domain=', ';', 'public', 'netstat', 'z9', '\\x85\\x140\\xfa\\x00', '[', 'z0', 'z1', 'z2', 'alvarezmaria', '\\xfe\\x1bf\\x97', "\\'posix_getegid\\", 'ini_set', '\\xd7\\xaa\\xb5\\xbd', '\\r\\', '/wwwroot/timeweekly-xx/news/2017265265265.php', '{', 'teresa56', 'd,0,1', '\\xb6\\x17\\xa6z\\xcf\\xe8\\xae', '\\xf4\\xe3m\\xfc\\xe3m\\xfc\\xe0', '/index.php', 'substr', 'haorenge.comqq317275738', 'berrycourtney', '/media/', 'j\\x9aly\\xcb\\xab\\x8a\\xdc\\xb5', '&', 'x4eac', "\\'document_root\\", 'usr', '``', '\\xe8.\\xf8\\xe4\\x0ez\\xdf\\xde\\xf7\\xeb^', '\\xdf^\\xf4\\xeb\\x8e7\\xeb', 'name=', 'chrome=1', '//life-order.su/', 'ert', '2fp10v.php', 'tterry', 'v\\x04\\x10\\xf9\\x140w\\x03p\\xbb\\x10\\x10', 'doctype', '-d\\r\\nrewriterule', 'root', 'tar', '\\xdc-\\xb9\\xdb\\xb0\\xbc\\x13\\xb', 'joshua51', './ybixuty7/', 'strlen', '\\xf4\\xe0', '127.0.0.1', '!', 'end', 'ode\\\\\\\\\\\\', './mqolmmky/', 'r.=php_uname', 'ping', 'foreach', 'xhtml', '-an', 'www.baidu.com', '/wwwroot/timeweekly-xx/news/news31.php', "\\'http\\", 'description', 'buf.=urldecode', '|all|00999.asp', '/wwwroot/timeweekly-xx/news/20170319503/index.php', "\\'appl_physical_path\\", 'enctype', '\\r\\n\\t', 'qfoster', 'charset=gb2312', '/alidata/www/home/htdocs/site/./lao.ph', 'x535a', '/alidata/www/home/htdocs/site//nuokangwen.ph', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\bbs\\\\\\\\aceior.php', '\\x9fr\\xdb\\x05\\xa9\\xd4b\\x16\\xf70\\xf9', 'opts', '\\xc2', 'delfile', '\\r\\n\\r\\necho', '\\\\\\\\wwwroot\\\\\\\\lijiang\\\\\\\\admin\\\\\\\\php_config.php', '/wwwroot/timeweekly-xx/news/2017373737.php', 'remote_server.', '\\x07_\\x0e\\x15\\x03^', '.substr', '\\r\\n', 'fwrite', ',', 'set_magic_quotes_runtime', 'x620f', '<', 'jiami.php', '\\xe7\\x99\\xbb\\xe5\\xbd\\x95\\xe7\\xb3\\xbb\\xe7\\xbb\\x9f', 'd==', '4msf.php', 'eval\\x01', 'php\\r\\nset_time_limit', 'execute', 'gethttppage', 'l.', '/ifmodule', '\\x00', 'filesman', '|', 'k\\xff\\xff\\xff\\xff\\xff\\xff\\xff\\xff\\xc', 'meta', '\\xfe\\xeb+\\xfe\\xc8z\\xad\\xef\\xe7\\x8', '/wwwroot/timeweekly-xx/news/20170319939/index.php', 'x5317', '/data/wwwroot/yujia/xiseceshi.ht', '\\\\gonghui\\\\', '/alidata/www/home/htdocs/site/css/haolanna.ph', 'loginlogin', 'print', 'eval/**/\\x01', 'ass', '\\xb4\\xe0', 'cawer1qw351gasdqa5rhgq', 'wscript', 'xt0jh8am', 'ug0epdgi', './pz4lu1ap/', 'x5f69', '/alidata1/www.cyxr.com//index.htm', 'http-equiv=', '/www/tengzhou0632/xinwen/jsw_87.html', 'xx=\\\\\\\\\\\\', 'r=', 'gb2312', '5\\xe7\\xbe\\xb7\\xef\\xf8m\\x9a_\\xf0\\xa7\\xfc', '\\xfd\\xd6\\xadk\\xfc0\\xc3\\xf8m\\x9a_\\xed\\xad\\xa', '\\x02\\x149y\\x1a-\\x00', 'bprice', 'x8d4c', '/proc/self/environ', 'x624b', '\\xc0\\x9a', 'xmlns=', '/wwwroot/dnwcomcn/link/xiongxian.php', 'echo', 'get_current_user', 'array', 'mkdir', 'windows-125\\\\00', 'white', 'google|aol|yahoo|msn|search|bing|seznam|seznam', 'filesize', 'function_exists', 'shell', 'jstephens', 'para', '2.fires', 'suzanne76', "\\'path_translated\\", '../mp3/v02.mp3', '\\xebj\\xbc', 'robots.txt', '\\xf4\\xe3m\\xfc\\xe3m\\xfc\\xe3m\\xfc\\xe3m\\xfc\\xe0', "_p'.'os", 'melissacalhoun', 'ie=edge', 'path_translated', "\\'a100a2016a\\", '1/', 'x5e73', 'and', '\\xfa\\x1f~\\xc8h\\xaf\\xfannx\\x9c\\xfe\\x1bf\\x97', 'response.write', 'content_mb=gethttppage', '^', 'haoren', 'phpfunc', '/wwwroot/timeweekly-xx/news/20170319107.php', '_po', '-', '/home/webapps/hdxxt/22.tar.gz', '/wwwroot/timeweekly-xx/news/20170319459/index.php', '=', '-zcvf', 'begin', 'x5e01', '/home/u150714/solncetut.ru/www/images/stories/', '/home/f/felixalarmes/www/', 'content=', 'net', 'mod_rewrite.c', 'content_mb=file_get_contents', '\\\\\\\\\\\\', 'buf=', 'dom', 'url', 'x53d1', '/home/users/s/sergey-it7/domains/south-hotel.ru/', '\\\\gygolf\\\\', '\\x99\\xe3*/\\x01', '/wwwroot/timeweekly-xx/html//201703154/index.php', '}', '21|23|25|80|110|135|139|445|1433|3306|3389|43958', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\hscd\\\\\\\\yhtjwrx.asp', '/wwwroot/timeweekly-xx/news/20170319486/index.php', 'x7406', './awthbf9o/', 'feliciabailey', 'x7403', 'keywords', './ieqven8z/', 'mkfile', 'parameters', 'title', '(', '163.com', '_server', '\\\\\\\\btc_web_yunxian\\\\\\\\images\\\\\\\\tab', '/hostingspaces/spdtrust/spdtrust.org/wwwroot/wow', 'pk1', '\\x0by', 'get', "\\'server_name\\", 'campbellaimee', '\\xfb\\xacz\\xbb', './szycho1o/', "\\'query_st", '\\xdc\\x96\\xe7^\\xb3', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\hsbk\\\\\\\\zksnuu.php', '/home/dveriminsk/www/dveri-minsk.by/', '/inetpub/wwwroot/spcm/plugins/', './rlbhkku3/', '\\r\\nheader', '\\xd7n\\xf7\\xf3\\x8d\\xfd\\xdb\\x9e\\xfa', 'transitional//en', 'x8f66', 'root=isset', '-//w3c//dtd', '.htaccess', 'x7f51', '\\\\n\\\\6q7.info', 'view', 'uu=', 'posix_getpwuid', 'set', 'alexandria15', '\\x17p\\xf8\\x08\\x10\\xf4\\x0f', '\\x85\\xdbnx\\xeb\\xce\\xbd\\xef', '\\x005\\x12', 'qdixon', '/data/home/qxu1606540025/htdocs/w', 'is_dir', '#', './cs04m9ky/', '\\xaez\\xbc\\xbf', '2017-03-27', '\\\\r', 'wordpress\\r\\n', '/wwwroot/timeweekly-xx/news/2017208208208.php', '\\\\t', '/www/tengzhou0632/xinwen/jsw_376.html', '\\\\n', '\\x9b\\xfb\\x9dj\\xd6\\xb6\\xf7', "\\'name\\", 'url=', 'x8d5b', 'c1sall', '/**/\\x01', 'jquery.js', 'se6', 'f=base64_decode', '/*\\xa0\\x9b', 'x4eba', '\\\\apm_setup\\\\server\\\\php4\\\\php.exe', 'remote_server', 'ifmodule', "\\'query_string\\", './hmyyjlho/', 'pk10', "\\'http", '/bs/apache-tomcat-kaihu/webapps/khp/', 'login', '\\xfd\\xda', '/wwwroot/timeweekly-xx/news/2017155155155.php', '\\x02g\\x03_\\x17\\x03u\\x16/5\\x0b_-\\x03r', "\\'\\", '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\bbs\\\\\\\\xutybybh.aspx', 'd=dirname', 'script_filename', 'chmod', 'i+=2', './31gitp7z/', '/data/www/default/app_api/data/reg_type.tx', 'array_map', 'usr=', '/ccmd', '81sq3zsa', '\\xdc-\\xc5\\xefn\\xbc\\xefm\\xb4\\xd', 'x8db3', '.', '/alidata/www/home/htdocs/site/ad/daoshui.ph', '>', '/add', 'qsa', '\\xe0\\x99', '\\x0b\\xc0', '//www.rarrar.pw/2/', 'wp-alone.php', '\\\\freehost\\\\zjgddkj01a\\\\web\\\\inc\\\\tu.jpg', 'host=', './lp583qcr/', 'main', 'downdb', 'p=', 'i,2', 'rename', '/data/wwwroot/www.sharknames.com/admin/sn_backup', 'http_user_agent', 'systeminfo', '\\x9d\\xa9\\x7fr\\xcb', 'melendezelizabeth', 'dennis26', 'gsalazar', 'base64_', '8-9999108099999', '\\x03\\x19s', '\\xcc\\xd7\\xfc', 'index.php', ')', '/d', '/c', '\\xfb\\xdb\\xbd\\xbd', "\\'prc\\", 'html', '/websoft/www/meilun/xiseceshi.phtml', '\\xfbazx\\xac\\xf3\\xfannx\\x9c\\xfe\\x1bf\\x97', './1ejfl7sa/', 'http', 'set_time_limit', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\hssp\\\\\\\\fjxfmq.php', '16:03:48', 'qoud8vam', 'user', '\\r\\n\\r\\nfunction', 'x6c11', '\\xe2\\x9e', '//', '/*', '/www/wt168', 'c=str_replace', '/web/immovision/wi/images/screen/', 'martinthomas', '/wwwroot/timeweekly-xx/news/20170319261/index.php', 'x4ee3', '-f\\r\\nrewritecond', '_post', 'base64_decode', '\\xec\\xff6\\x9b\\xa2\\xbf\\xaf\\xa2x\\x9e\\xae\\xbb\\xbf', 'x6e38', 'touch', 'd\\xe3\\x93\\xfd\\xb6\\xa4\\x8e\\xca', './ibgod5bq/', '\\r\\ndate_default_timezone_set', '$', '/wwwroot/timeweekly-xx/news/20170319556/index.php', 'location', '\\xf1\\xfe\\xbc\\xf3\\xb3\\x9e\\xbf\\x96', '//119.28.15.141/news/', './4ojik8mp/', '\\xd6k\\xa5\\xb6', '16:30:01', '\\xa1\\xcb', 'utf-8', '../', '...', '\\xf4\\xe8^\\xf6\\xef\\x8d\\xb4\\xe0n\\xb5\\xe8', 'php_self', 'x4f17', '\\\\', '9xthzpbs', '\\xb3\\xfb\\x1e\\xae\\x07\\xb2\\xfa+', 'request_filename', 'nlm2ig2b', '../web.config', '.fires', '\\x04\\x1fq-f', '2015-03-22', '/godyeweb/mpa1/uc_server/install/xiseceshi.cfm', 'text/html', 'header', 'ly81nzm3nc5odg1s', '53=53', 'edit', '/\\r\\nrewriterule', 'hdklqbqyiiixom0o', '//shxysh.com/', '\\xfb', '\\xfa', 'posix_geteuid', '/pleskvhosts/rbagroups.com/vitspoly.org/a/j/', '\\x178', 'hhunter', 'taylorjessica', '\\r\\n\\r\\nheader', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\public\\\\\\\\spwkkvwb.php', 'for', '/home/wwwroot/encopper/public_html/catalog/model/', '/', '\\x89\\xd7\\xab\\xdf\\xfb', "\\'http_host\\", '?', 'run', './ywilface/', '\\xf4\\xe3m\\xfc\\xe0', '/home/webapps/hdxxt', '\\\\\\\\', '\\xf7', 'r.=', 'smithrickey', 'isset', '_', '\\xd3m4\\xdf\\xc', 'i=0', '\\xfe\\x1', '\\xb4\\xeb\\xae\\xbd\\xe8.\\xb9\\xdbn\\xbd\\xef', '../web-inf/web.xml', './kuqo2b0g/', 'range', 'content-type', 'x9645', '1999999yyyy', '/alidata/www/home/htdocs/site//fenchuozhang.ph', 'visible', '\\xff\\xfc', '/*12345*/base64_decode', 'd_m2ps.php', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\api\\\\\\\\spwkkvwb.php', 'fopen', 'u=', 'on\\r\\nrewritecond', '.*', 'display_errors', ']', '*', 'x56fd', 'wordpress', 'amandafreeman', ':', '4_dec', 'head', 'pwd', 'eval/*\\x15\\x99\\xd0', '/alidata/www/home/htdocs/site//chengmi.ph', 'iduke', 'buf', '^index\\\\.php', 'edoced_46esab', '\\r\\nrewriteengine', '-c/bin/sh', 'trim', '\\r\\n\\r\\n', '//103.224.249.50/', 'fread', '\\xe7\\xae\\xbc\\xe4\\x0e', 'john20', '%', 'http_referer', 'xiseceshineiron', 'i\\xdb\\xb0jw', 'garnold', 'x-ua-compatible', '\\\\\\\\wwwroot\\\\\\\\haishen\\\\\\\\statics\\\\\\\\aceior.php', 'str_replace', 'x620', '\\x07\\x1b=', '\\x9e\\xd7\\xa7\\xb7\\xfd\\xfb\\xff', 'eval', '\\x00\\x00', '\\xc0\\xdf', 'zoeanytm', 'downfile', '\\xdc\\r\\xf5\\xdf\\xbd\\xfb\\xdf', '\\xfa\\xdarf\\xa7v\\xbf\\xe9\\xb9\\xb9bs\\xf8m\\x9a_']


doc_features_matrix = get_doc_features_matrix(doc_word_list, features)

In [25]:
X_train=doc_features_matrix
y_train=y
X_test=test_data

In [27]:
clf=tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)
result=clf.predict(X_test)

In [40]:
i=0
for item in result:
    if item==1:
        i=i+1
print (i)

3928


250000


In [46]:
i=0
f=open(r'C:\Users\pc1\Desktop\result.txt','w',encoding='utf-8')
f.write('ID'+','+'FLAG'+'\n')
for item in result:
    if item==1:
        f.write(str(i+1250521)+','+'w'+'\n')
    i=i+1
f.close()
print (i)

250000
